In [ ]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import random
import string
import os
from datetime import datetime

class CaptchaConfig:
    """验证码生成配置参数类"""
    def __init__(self):
        # 图像尺寸
        self.width = 120
        self.height = 45
        
        # 字体配置
        self.font_paths = [
            'arial.ttf',
            os.path.join(os.path.dirname(os.getcwd()), 'fonts/arial.ttf'),
            '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf',
            '/Library/Fonts/Arial.ttf',
            'C:/Windows/Fonts/arial.ttf'
        ]
        self.font_size_range = (24, 32)     # 字体大小范围
        
        # 验证码内容配置
        self.char_count = 4                 # 字符数量
        self.char_set = string.digits + string.ascii_letters  # 使用的字符集
        
        # 干扰元素配置
        self.initial_noise = 30             # 初始噪点数量
        self.initial_lines = 3              # 初始干扰线数量
        self.final_noise = 50               # 最终噪点数量
        self.final_lines = 2                # 最终干扰线数量
        
        # 视觉效果参数
        self.rotation_range = (-30, 30)     # 字符旋转角度范围
        self.offset_range = (-5, 5)         # 字符位置偏移范围
        self.color_range = (50, 150)        # 字符颜色RGB分量范围
        self.bg_color = (255, 255, 255)     # 背景颜色
        
        # 输出配置
        self.output_dir = os.path.join(os.path.dirname(os.getcwd()), 'data/generate')
        self.file_suffix = 'png'            # 文件格式


def generate_captcha(config=CaptchaConfig()):
    """生成验证码并保存到文件"""
    # 初始化画布
    image = Image.new('RGB', (config.width, config.height), config.bg_color)
    draw = ImageDraw.Draw(image)

    try:
        # 加载字体文件
        font_path = _load_font(config.font_paths)
        
        # 生成随机验证码
        captcha_text = _generate_random_string(config.char_count, config.char_set)
        
        # 添加初始干扰元素
        _add_noise(draw, config.width, config.height, config.initial_noise)
        _add_interference_lines(draw, config.width, config.height, config.initial_lines)

        # 绘制验证码字符
        for idx, char in enumerate(captcha_text):
            char_img = _create_char_image(
                char=char,
                font_path=font_path,
                font_size=random.randint(*config.font_size_range),
                color=(
                    random.randint(*config.color_range),
                    random.randint(*config.color_range),
                    random.randint(*config.color_range)
                ),
                rotation=random.randint(*config.rotation_range)
            )
            
            # 计算字符位置
            x_pos = idx * (config.width / config.char_count)
            x_offset = random.randint(*config.offset_range)
            y_offset = random.randint(*config.offset_range)
            position = (
                int(x_pos + (config.width/config.char_count - char_img.width)/2 + x_offset),
                int((config.height - char_img.height)/2 + y_offset)
            )
            
            # 合并字符图像
            image.paste(char_img, position, char_img)

        # 添加最终干扰元素
        _add_noise(draw, config.width, config.height, config.final_noise)
        _add_interference_lines(draw, config.width, config.height, config.final_lines)
        image = image.filter(ImageFilter.SMOOTH)

        # 保存文件
        filename = _generate_filename(captcha_text, config.file_suffix)
        os.makedirs(config.output_dir, exist_ok=True)
        output_path = os.path.join(config.output_dir, filename)
        image.save(output_path, quality=95)
        
        return output_path, captcha_text
    except Exception as e:
        raise RuntimeError(f"验证码生成失败: {str(e)}") from e


def _load_font(font_paths):
    """加载第一个可用的字体文件"""
    for path in font_paths:
        if os.path.exists(path):
            return path
    raise FileNotFoundError(f"未找到可用字体，尝试路径: {font_paths}")


def _generate_random_string(length, char_set):
    """生成随机字符串"""
    return ''.join(random.choices(char_set, k=length))


def _create_char_image(char, font_path, font_size, color, rotation):
    """创建单个字符的旋转图像"""
    font = ImageFont.truetype(font_path, font_size)
    
    # 创建足够大的临时画布
    temp_img = Image.new('RGBA', (font_size*3, font_size*3), (0, 0, 0, 0))
    temp_draw = ImageDraw.Draw(temp_img)
    
    # 计算居中位置
    bbox = temp_draw.textbbox((0, 0), char, font=font)
    position = (
        (temp_img.width - (bbox[2] - bbox[0]))/2 - bbox[0],
        (temp_img.height - (bbox[3] - bbox[1]))/2 - bbox[1]
    )
    
    # 绘制并旋转字符
    temp_draw.text(position, char, font=font, fill=color)
    return temp_img.rotate(rotation, expand=True, fillcolor=(0, 0, 0, 0))


def _add_noise(draw, width, height, density):
    """添加随机噪点"""
    for _ in range(density):
        draw.point(
            (random.randint(0, width-1), random.randint(0, height-1)),
            fill=(random.randint(100, 255), random.randint(100, 255), random.randint(100, 255))
        )


def _add_interference_lines(draw, width, height, num_lines):
    """添加干扰线"""
    for _ in range(num_lines):
        points = [
            (random.randint(0, width//2), random.randint(0, height)),
            (random.randint(width//2, width), random.randint(0, height))
        ]
        draw.line(
            points,
            fill=(random.randint(50, 200), random.randint(50, 200), random.randint(50, 200)),
            width=random.randint(1, 2)
        )


def _generate_filename(text, suffix):
    """生成带时间戳的文件名"""
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    random_str = ''.join(random.choices(string.digits, k=4))
    return f"{text}_{timestamp}{random_str}.{suffix}"



# 自定义配置
custom_config = CaptchaConfig()

# 生成验证码
for i in range(4900):
    file_path, text = generate_captcha(custom_config)

In [15]:
string.digits + string.ascii_letters

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'